In [27]:
from importlib import import_module
from pprint import pprint

from deep_image_matching import logger, timer
from deep_image_matching.config import Config
from deep_image_matching.image_matching import ImageMatcher
from deep_image_matching.io.h5_to_db import export_to_colmap

import yaml

#### Define the configuration

Get the list of possible configurations and chose one of them.

In [ ]:
print("Available configurations:")
pprint(Config.get_pipelines())
print("Available matching strategy:")
pprint(Config.get_matching_strategies())

Now you have to build a dictionary with the input processing parameters (they are the same as the input parameters for the CLI and GUI) and pass it to the Config class to initialize the configuration object.
Refer to the README for more information about the parameters.

Note that there are two possible approaches for defining the paths needed for the processing (i.e., input images and output results):
- You can pass a single parameter defining the processing directory (with 'dir' key). Deep-Image-Matching will search for the images inside an 'image' subdirectory and will save the results in a 'results_{processing_params}' subdirectory, where {processing_params} are some information on the processing parameters used.
- or you can manually specify the input images directory (with 'images' key) and the output results directory (with 'outs' key).

Note, that these parameters are the same as the ones used in the CLI (the GUI is not updated yet).

In [29]:
cli_params = {
    "dir": "../assets/example_cyprus",
    "pipeline": "superpoint+lightglue",
    "strategy": "matching_lowres",
    "quality": "high",
    "tiling": "preselection",
    "skip_reconstruction": False,
    "force": True,
    "camera_options": "../config/cameras.yaml",
    "openmvg": None,
}
config = Config(cli_params)

You can check the configuration object.

In [ ]:
print("Config general:")
pprint(config.general)
print("Config extractor:")
pprint(config.extractor)
print("Config matcher:")
pprint(config.matcher)

If you know what you are doing, you can update some config parameters directly updating the config dictionary (check the file config.py in the scr folder for the available parameters).


In [31]:
# - General configuration
config.general["min_inliers_per_pair"] = 10
config.general["min_inlier_ratio_per_pair"] = 0.2

# - SuperPoint configuration
config.extractor["max_keypoints"] = 8000

# - LightGue configuration
config.matcher["filter_threshold"] = 0.1

# Save configuration to a json file in the output directory
config.save()

In [32]:
imgs_dir = config.general["image_dir"]
output_dir = config.general["output_dir"]
matching_strategy = config.general["matching_strategy"]
extractor = config.extractor["name"]
matcher = config.matcher["name"]

#### Initialize the ImageMatcher class 
This will be used for performing the image matching.

In [ ]:
img_matching = ImageMatcher(
    imgs_dir=imgs_dir,
    output_dir=output_dir,
    matching_strategy=matching_strategy,
    local_features=extractor,
    matching_method=matcher,
    pair_file=config.general["pair_file"],
    retrieval_option=config.general["retrieval"],
    overlap=config.general["overlap"],
    existing_colmap_model=config.general["db_path"],
    custom_config=config.as_dict(),
)

#### Generate pairs to be matched

In [ ]:
pair_path = img_matching.generate_pairs()
timer.update("generate_pairs")

Try to rotate images so they will be all "upright", useful for deep-learning approaches that usually are not rotation invariant

In [35]:
if config.general["upright"]:
    img_matching.rotate_upright_images()
    timer.update("rotate_upright_images")

#### Extract features

In [ ]:
feature_path = img_matching.extract_features()
timer.update("extract_features")

#### Run matching

In [ ]:
match_path = img_matching.match_pairs(feature_path)
timer.update("matching")

If features have been extracted on "upright" images, this function bring features back to their original image orientation

In [38]:
if config.general["upright"]:
    img_matching.rotate_back_features(feature_path)
    timer.update("rotate_back_features")

#### Export in colmap format

DIM assigns camera models to images based on the options defined in `config/cameras.yaml`. To load camera model options:

In [ ]:
with open(config.general["camera_options"], "r") as file:
    camera_options = yaml.safe_load(file)

Alternatively you can assign camera models with a dictionary. 

For images not assigned to specific `cam<x>` camera groups, the options specified under `general` are applied. The `camera_model` can be selected from `["simple-pinhole", "pinhole", "simple-radial", "opencv"]`. It's worth noting that it's easily possible to extend this to include all the classical COLMAP camera models. Cameras can either be shared among all images (`single_camera == True`), or each camera can have a different camera model (`single_camera == False`).

A subset of images can share intrinsics using `cam<x>` key, by specifying the `camera_model` along with the names of the images separated by commas. There's no limit to the number of `cam<x>` entries you can use.

**Note**: Use the SIMPLE-PINHOLE camera model if you want to export the solution to Metashape later, as there are some bugs in COLMAP (or pycolamp) when exportingthe solution in the Bundler format.
e.g., using FULL-OPENCV camera model, the principal point is not exported correctly and the tie points are wrong in Metashape.

In [ ]:

camera_options = {
   'general' : {
    "camera_model" : "pinhole", # ["simple-pinhole", "pinhole", "simple-radial", "opencv"]
    "single_camera" : True,
   },
   'cam0' : {
        "camera_model" : "pinhole",
        "images" : "DSC_6468.JPG,DSC_6468.JPG",
   },
   'cam1' : {
        "camera_model" : "pinhole",
        "images" : "",
   },
}
    
database_path = output_dir / "database.db"
export_to_colmap(
    img_dir=imgs_dir,
    feature_path=feature_path,
    match_path=match_path,
    database_path=database_path,
    camera_options=camera_options,
)
timer.update("export_to_colmap")

In [46]:
print(camera_options)

{'general': {'camera_model': 'pinhole', 'single_camera': True}, 'cam0': {'camera_model': 'pinhole', 'images': 'DSC_6468.jpg,DSC_6468.jpg'}, 'cam1': {'camera_model': 'pinhole', 'images': ''}}


#### Run reconstruction
If --skip_reconstruction is not specified (from CLI or in the cli_params dictonary), run reconstruction with pycolmap

Try first to import the pycolmap module, if it fails, skip reconstruction



In [47]:
if not config.general["skip_reconstruction"]:
    use_pycolmap = True
    try:
        pycolmap = import_module("pycolmap")
    except ImportError:
        logger.error("Pycomlap is not available.")
        use_pycolmap = False

If the pycolmap module is imported, define some parameters for the reconstruction and run it.
(Optional) You can specify some reconstruction configuration in a dictionary, or leave the dictionary empty to use the default configuration.

  ``` python  
  reconst_opts = {
          "ba_refine_focal_length": True,
          "ba_refine_principal_point": False,
          "ba_refine_extra_params": False,
      }
  ```
  or
  ``` python
  reconst_opts = {}
  ```

In [ ]:
if not config.general["skip_reconstruction"] and use_pycolmap:
    # import reconstruction module
    from deep_image_matching import reconstruction

    # Define database path and camera mode
    database = output_dir / "database.db"
    camera_mode = pycolmap.CameraMode.AUTO
    cameras = None
    reconst_opts = {}
    model = reconstruction.main(
        database=database,
        image_dir=imgs_dir,
        feature_path=feature_path,
        match_path=match_path,
        pair_path=pair_path,
        sfm_dir=output_dir,
        camera_mode=camera_mode,
        cameras=cameras,
        skip_geometric_verification=True,
        reconst_opts=reconst_opts,
        verbose=config.general["verbose"],
    )

    timer.update("pycolmap reconstruction")


In [ ]:
# Print COLMAP camera values
print(list(model.cameras.values()))

#### Import the solution in Metashape

If the reconstruction with pycolmap has been performed, you can import the solution in Metashape.
This is done by first exporting the solution in Bundler format and then importing it in Metashape.

This can be performed automatically with the function `export_to_metashape()`, which can also run a Bundle Adjustment.

**Note** that this function is under development and it is not yet integrated in Deep-Image-Matching (but it is inside a script in the scripts directory). However, you can use it as an example to export the solution to Metashape.

In [ ]:
# import the function from the script folder
from scripts.metashape.metashape_from_dim import export_to_metashape


# Define the paths for the ne Metashape project and the path of the Bundler filter
project_dir = config.general["output_dir"] / "metashape"
project_name = config.general["output_dir"].name + ".psx"
project_path = project_dir / project_name

rec_dir = config.general["output_dir"] / "reconstruction"
bundler_file_path = rec_dir / "bundler.out"
bundler_im_list = rec_dir / "bundler_list.txt"


# Define the interior orientation parameters to refine or fix during the bundle adjustment (the parameters are the same as in the Metashape GUI)

prm_to_optimize = {
    "f": True,
    "cx": True,
    "cy": True,
    "k1": True,
    "k2": True,
    "k3": True,
    "k4": False,
    "p1": True,
    "p2": True,
    "b1": False,
    "b2": False,
    "tiepoint_covariance": True,
}

# Export the reconstruction to Metashape
export_to_metashape(
    project_path=project_path,
    images_dir=config.general["image_dir"],
    bundler_file_path=bundler_file_path.resolve(),
    bundler_im_list=bundler_im_list.resolve(),
    prm_to_optimize=prm_to_optimize,
)

In [ ]:
timer.print()